# elaws-openapi-proxy example usage by langchain

## Prerequisites

1. This example uses the [OpenAI API](https://openai.com/blog/openai-api). Please prepare your OpenAI API key. Be aware that calling the API may incur costs.

2. Get ready your [elaws-openapi-proxy server](../../README.md) and make sure the OpenAPI spec of the server is accessible from the environment this notebook runs.

3. The Python environment of this notebook is managed by [Poetry](https://python-poetry.org/docs/#installation). Please install Poetry if necessary, and run `poetry install` in the same directory as this notebook.

## How to run the notebook

1. Save the OpenAI API key in the [`.env`](.env) file in the same directory as this notebook. Write the key in the `.env` file as below.
    ```
    OPENAI_API_KEY=sk-*******************************
    ```

2. Before running the notebook, make sure the [elaws-openapi-proxy server](../../README.md) is running. If necessary, please modify the `ELAWS_OPENAPI_SPEC_URI` variable in the next cell.

3. Run the all cells below.

In [1]:
"""
The OpenAPI spec URI of the elaws-openapi-proxy server.
Please modify it if necessary.
"""
ELAWS_OPENAPI_SPEC_URI = "http://localhost:3000/openapi.json"

In [2]:
from dotenv import load_dotenv
assert load_dotenv()
import langchain
print(f"{langchain.__version__ = }")

from langchain.chains.openai_functions.openapi import get_openapi_chain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.prompts import ChatPromptTemplate

langchain.__version__ = '0.0.240'


In [3]:
llm = ChatOpenAI(
    model="gpt-4-0613",
    temperature=0,
    verbose=True,
)
chain = get_openapi_chain(
    spec=ELAWS_OPENAPI_SPEC_URI,
    llm=llm,
    verbose=True,
    prompt=ChatPromptTemplate.from_template(
        "Keep in mind that the exact words and characters matter for laws. Do not make up an answer. Use the provided functions to respond to this user query:\n\n{query}"
    )
)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [4]:
tools = [
    Tool(
        name="Law-DB",
        func=chain.run,
        description="useful for when you need to answer questions about laws",
    ),
]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

In [5]:
output = agent.run("デジタル庁設置法（令和三年法律第三十六号）第一条の内容を教えてください。")
print(output)



> Entering new AgentExecutor chain...

Invoking: `Law-DB` with `デジタル庁設置法（令和三年法律第三十六号）第一条`




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Keep in mind that the exact words and characters matter for laws. Do not make up an answer. Use the provided functions to respond to this user query:

デジタル庁設置法（令和三年法律第三十六号）第一条

> Finished chain.


> Entering new SimpleRequestChain chain...
Calling endpoint GetArticles with arguments:
{
  "params": {
    "lawNum": "\u4ee4\u548c\u4e09\u5e74\u6cd5\u5f8b\u7b2c\u4e09\u5341\u516d\u53f7",
    "article": "1"
  }
}
> Finished chain.

> Finished chain.
{'Result': {'Code': '0', 'Message': ''}, 'ApplData': {'LawId': '', 'LawNum': '令和三年法律第三十六号', 'Article': '1', 'Paragraph': '', 'AppdxTable': '', 'LawContents': '\n      <Article Num="1">\n        <ArticleCaption>（目的）</ArticleCaption>\n        <ArticleTitle>第一条</ArticleTitle>\n        <Paragraph Num="1">\n          <ParagraphNum/>\n          <Paragrap